In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler, Imputer
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats

In [2]:
#Read in Data
train = pd.read_csv('dataFiles/application_train.csv')
test = pd.read_csv('dataFiles/application_test.csv')
bureau_data = pd.read_csv('dataFiles/bureau.csv')
bureau_balance_data = pd.read_csv('dataFiles/bureau_balance.csv')
prev_app_data = pd.read_csv('dataFiles/previous_application.csv')
pos_cash_balance_data = pd.read_csv('dataFiles/POS_CASH_balance.csv')

In [3]:
pos_cash_balance_data_grouped = pos_cash_balance_data.groupby('SK_ID_PREV', as_index=False).agg({'MONTHS_BALANCE':min,'CNT_INSTALMENT':'median',
                                                                 'CNT_INSTALMENT_FUTURE':'median','CNT_INSTALMENT_FUTURE':'median',
                                                                'SK_DPD':sum, 'SK_DPD_DEF':sum})

In [4]:
prev_app_data_merged = prev_app_data.merge(pos_cash_balance_data_grouped, how = 'left', left_on = 'SK_ID_PREV', right_on = 'SK_ID_PREV')

In [5]:
bureau_balance_data_grouped = pd.get_dummies(bureau_balance_data).groupby('SK_ID_BUREAU', as_index=False).agg({'MONTHS_BALANCE':min, 'STATUS_0':sum,
                                                                                'STATUS_1':sum, 'STATUS_2':sum, 'STATUS_3':sum,
                                                                                'STATUS_3':sum, 'STATUS_4':sum, 'STATUS_5':sum,
                                                                                'STATUS_5':sum, 'STATUS_X':sum})

In [6]:
bureau_data = bureau_data.merge(bureau_balance_data_grouped, how = 'left', left_on = 'SK_ID_BUREAU', right_on = 'SK_ID_BUREAU')

In [7]:
# Joining Bureau Grouped Data with Train Data
bureau_data_grouped = bureau_data.groupby('SK_ID_CURR', as_index=False).agg({'SK_ID_BUREAU': 'count', 'AMT_CREDIT_MAX_OVERDUE':sum,\
                                                                             'AMT_CREDIT_SUM_OVERDUE': sum, 'DAYS_CREDIT': 'mean',\
                                                                             'CREDIT_DAY_OVERDUE': sum, 'DAYS_CREDIT_ENDDATE': 'mean',\
                                                                             'AMT_CREDIT_SUM': sum, 'AMT_CREDIT_SUM_DEBT': sum,\
                                                                             'MONTHS_BALANCE':min, 'STATUS_0':sum,\
                                                                             'STATUS_1':sum, 'STATUS_2':sum, 'STATUS_3':sum,\
                                                                             'STATUS_3':sum, 'STATUS_4':sum, 'STATUS_5':sum,\
                                                                             'STATUS_5':sum, 'STATUS_X':sum
                                                                            })

train_merged = train.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

test_merged = test.merge(bureau_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [8]:
prev_app_data_subset = prev_app_data_merged[['SK_ID_CURR','AMT_ANNUITY','AMT_APPLICATION','AMT_CREDIT','AMT_DOWN_PAYMENT','AMT_GOODS_PRICE',\
                         'CNT_PAYMENT','DAYS_FIRST_DRAWING','DAYS_FIRST_DUE','DAYS_LAST_DUE_1ST_VERSION',\
                         'DAYS_LAST_DUE','DAYS_TERMINATION','NFLAG_INSURED_ON_APPROVAL','MONTHS_BALANCE','CNT_INSTALMENT',\
                                             'CNT_INSTALMENT_FUTURE', 'CNT_INSTALMENT_FUTURE', 'SK_DPD', 'SK_DPD_DEF']]

In [9]:
prev_app_data_subset.columns = ['SK_ID_CURR','AMT_ANNUITY_PRE','AMT_APPLICATION_PRE','AMT_CREDIT_PRE','AMT_DOWN_PAYMENT_PRE','AMT_GOODS_PRICE_PRE',\
                         'CNT_PAYMENT_PRE','DAYS_FIRST_DRAWING_PRE','DAYS_FIRST_DUE_PRE','DAYS_LAST_DUE_1ST_VERSION_PRE',\
                         'DAYS_LAST_DUE_PRE','DAYS_TERMINATION_PRE','NFLAG_INSURED_ON_APPROVAL_PRE','MONTHS_BALANCE_PRE','CNT_INSTALMENT_PRE',\
                                             'CNT_INSTALMENT_FUTURE_PRE', 'CNT_INSTALMENT_FUTURE_PRE', 'SK_DPD_PRE', 'SK_DPD_DEF_PRE']

In [10]:
prev_app_data_grouped = prev_app_data_subset.groupby('SK_ID_CURR', as_index=False).sum()

In [11]:
train_merged = train_merged.merge(prev_app_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

test_merged = test_merged.merge(prev_app_data_grouped, how = 'left', left_on = 'SK_ID_CURR', right_on = 'SK_ID_CURR')

In [12]:
own_car_median = train_merged.OWN_CAR_AGE.median()

In [13]:
train_dummies = pd.get_dummies(train_merged[['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']], drop_first= True)

test_dummies = pd.get_dummies(test_merged[['FLAG_OWN_CAR', 'FLAG_OWN_REALTY']], drop_first = True)

In [14]:
def own_car_missing(x):
    if x['FLAG_OWN_CAR'] == 'N':
        return 28
    elif x['FLAG_OWN_CAR'] == 'Y' and pd.isnull(x['OWN_CAR_AGE']):
        return own_car_median
    else:
        return x['OWN_CAR_AGE']

In [15]:
train_merged = pd.concat([train_merged, train_dummies], axis = 1).select_dtypes(exclude=['object'])

test_merged = pd.concat([test_merged, test_dummies], axis = 1).select_dtypes(exclude=['object'])

In [16]:
imputer = Imputer(strategy = 'median')
imputer.fit(train_merged.drop('TARGET', axis = 1))
train_merged_imputed = pd.DataFrame(imputer.transform(train_merged.drop('TARGET', axis = 1)), columns = train_merged.drop('TARGET', axis = 1).columns)
test_merged_imputed = pd.DataFrame(imputer.transform(test_merged), columns = train_merged.drop('TARGET', axis = 1).columns)

In [52]:
train_merged_imputed['DAYS_EMPLOYED_^2'] = train_merged_imputed['DAYS_EMPLOYED'] ** 2
train_merged_imputed['AMT_GOODS_PRICE_^2'] = train_merged_imputed['AMT_GOODS_PRICE'] ** 2
train_merged_imputed['DAYS_CREDIT_^2'] = train_merged_imputed['DAYS_CREDIT'] ** 2
train_merged_imputed['DAYS_BIRTH_^2'] = train_merged_imputed['DAYS_BIRTH'] ** 2

test_merged_imputed['DAYS_EMPLOYED_^2'] = test_merged_imputed['DAYS_EMPLOYED'] ** 2
test_merged_imputed['AMT_GOODS_PRICE_^2'] = test_merged_imputed['AMT_GOODS_PRICE'] ** 2
test_merged_imputed['DAYS_CREDIT_^2'] = test_merged_imputed['DAYS_CREDIT'] ** 2
test_merged_imputed['DAYS_BIRTH_^2'] = test_merged_imputed['DAYS_BIRTH'] ** 2

In [53]:
column_corr = pd.concat([train_merged_imputed, train['TARGET']], axis = 1).corr()['TARGET'].sort_values()
#column_corr = column_corr[(column_corr > .02) | (column_corr < -.02)].index.values.tolist()

In [34]:
#column_corr = train_merged.corr()['TARGET'].sort_values()
#column_corr = column_corr[(column_corr > .01) | (column_corr < -.01)].index.values.tolist()
#column_corr.remove('TARGET')

In [35]:
#train_subset = train_merged[column_corr]

#test_subset = test_merged[column_corr]

In [55]:
poly_transformer = PolynomialFeatures(degree = 1)
poly_transformer.fit(train_merged_imputed)
train_poly_features = poly_transformer.transform(train_merged_imputed)

In [56]:
train_subset_poly = pd.DataFrame(train_poly_features, columns = poly_transformer.get_feature_names(
    input_features = train_merged_imputed.columns.tolist()
))

In [57]:
test_poly_features = poly_transformer.transform(test_merged_imputed)

In [58]:
test_subset_poly = pd.DataFrame(test_poly_features, columns = poly_transformer.get_feature_names(input_features = train_merged_imputed.columns.tolist()))

In [59]:
scaler = MinMaxScaler(feature_range = (0, 1))

In [60]:
scaler.fit(train_subset_poly)
train_scaled = scaler.transform(train_subset_poly)
test_scaled = scaler.transform(test_subset_poly)

In [61]:
from catboost import CatBoostClassifier
cat_model = CatBoostClassifier(iterations=500, learning_rate=0.05)

In [67]:
cat_model.fit(train_scaled, train.TARGET)

0:	learn: 0.6373116	total: 428ms	remaining: 3m 33s
1:	learn: 0.5885586	total: 892ms	remaining: 3m 42s
2:	learn: 0.5468374	total: 1.39s	remaining: 3m 50s
3:	learn: 0.5103647	total: 1.85s	remaining: 3m 49s
4:	learn: 0.4788976	total: 2.29s	remaining: 3m 46s
5:	learn: 0.4515853	total: 2.75s	remaining: 3m 46s
6:	learn: 0.4274958	total: 3.19s	remaining: 3m 44s
7:	learn: 0.4070092	total: 3.66s	remaining: 3m 45s
8:	learn: 0.3887973	total: 4.18s	remaining: 3m 48s
9:	learn: 0.3729272	total: 4.88s	remaining: 3m 59s
10:	learn: 0.3590346	total: 5.4s	remaining: 4m
11:	learn: 0.3474372	total: 5.94s	remaining: 4m 1s
12:	learn: 0.3373281	total: 6.43s	remaining: 4m
13:	learn: 0.3281266	total: 6.97s	remaining: 4m 1s
14:	learn: 0.3200603	total: 7.54s	remaining: 4m 3s
15:	learn: 0.3125186	total: 8.12s	remaining: 4m 5s
16:	learn: 0.3059704	total: 8.62s	remaining: 4m 4s
17:	learn: 0.2999568	total: 9.83s	remaining: 4m 23s
18:	learn: 0.2950654	total: 10.8s	remaining: 4m 33s
19:	learn: 0.2905463	total: 11.4s	re

In [68]:
test_y_cat = pd.DataFrame(cat_model.predict_proba(test_scaled))
submission_cat = pd.concat([test.SK_ID_CURR, test_y_cat], axis=1).drop(0, axis = 1)
submission_cat.columns = ['SK_ID_CURR', 'Target']

In [69]:
#max_sub_cat = submission_cat.Target.max()
#min_sub_cat = submission_cat.Target.min()

In [70]:
#submission_cat['Target'] = submission_cat['Target'].apply(lambda x: (x - min_sub_cat) / (max_sub_cat - min_sub_cat))

In [71]:
submission_cat.to_csv('reduced_cat14.csv', index = False)

In [ ]:
########################################################################

In [ ]:
column_corr = pd.concat([train_subset_poly, train['TARGET']], axis = 1).corr()['TARGET'].sort_values()
column_corr = column_corr.drop('TARGET')
column_corr = column_corr[(column_corr > .03) | (column_corr < -.03)].index.values.tolist()

In [ ]:
train_subset_poly = train_subset_poly[column_corr]

In [ ]:
test_subset_poly = test_subset_poly[column_corr]

In [ ]:
scaler.fit(train_subset_imputed)
train_scaled = scaler.transform(train_subset_imputed)
test_scaled = scaler.transform(test_subset_imputed)

In [ ]:
cat_model.fit(train_scaled, target)

In [ ]:
test_y_cat = pd.DataFrame(cat_model.predict_proba(test_scaled))
submission_cat = pd.concat([test.SK_ID_CURR, test_y_cat], axis=1).drop(0, axis = 1)
submission_cat.columns = ['SK_ID_CURR', 'Target']
submission_cat.to_csv('reduced_cat11.csv', index = False)